In [1]:
from graph.planner import planner

In [2]:
case = planner.invoke("Send 0.01 ETH to 0x8c575b178927fF9A70804B8b4F7622F7666bB360")
print(case)

ID: send_eth_example
Description: Send 0.01 ETH to a specified address on Ethereum.
Steps:
1. Transfer 0.01 ETH to address 0x8c575b178927fF9A70804B8b4F7622F7666bB360.


In [3]:
case = planner.invoke("Send 100 USDT to scott.")
print(case)

ID: send_usdt_to_scott
Description: Send 100 USDT to Scott.
Steps:
1. Transfer 100 USDT to Scott's Ethereum address.


In [4]:
case = planner.invoke("Swap 100 USDT to USDC")
print(case)

ID: swap_usdt_to_usdc
Description: Swap 100 USDT to USDC using Uniswap.
Steps:
1. Approve 100 USDT for Uniswap.
2. Swap 100 USDT to USDC on Uniswap.


In [5]:
case = planner.invoke("Stake 0.03 ETH with Lido and deposit to Eigenpie")
print(case)

ID: stake_eth_eigenpie
Description: Stake 0.03 ETH with Lido and deposit to Eigenpie
Steps:
1. Stake 0.03 ETH to Lido to receive stETH.
2. Approve stETH to be used by Eigenpie.
3. Stake stETH to Eigenpie.
